<a href="https://colab.research.google.com/github/makxxic/AI-for-SDevWK6/blob/main/AI4SDevWK6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("techsash/waste-classification-data")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'waste-classification-data' dataset.
Path to dataset files: /kaggle/input/waste-classification-data


In [2]:
import os

# The 'path' variable holds the base directory of the downloaded dataset.
# Its current value is: '/kaggle/input/waste-classification-data'
print(f"Inspecting the base dataset directory: {path}")

# List all contents (files and subdirectories) at the root of the downloaded dataset
print("Contents of the downloaded dataset:")
for item in os.listdir(path):
    full_path = os.path.join(path, item)
    if os.path.isdir(full_path):
        print(f"- {item}/ (Directory)")
        # Optionally, list contents of one level deeper if it seems relevant
        # if item == 'DATASET': # Example: if there's a 'DATASET' folder
        #     print(f"  Contents of {item}/")
        #     for sub_item in os.listdir(full_path):
        #         print(f"  - {sub_item}")
    else:
        print(f"- {item} (File)")


Inspecting the base dataset directory: /kaggle/input/waste-classification-data
Contents of the downloaded dataset:
- DATASET/ (Directory)
- dataset/ (Directory)


In [3]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Load Dataset
# The 'path' variable is set in the first cell (gEL9IIrhFh1k) by kagglehub.dataset_download
# Based on inspection, TRAIN and TEST folders are likely inside a 'DATASET' subdirectory.
train_dir = os.path.join(path, 'DATASET', 'TRAIN') # Corrected path
val_dir = os.path.join(path, 'DATASET', 'TEST')  # Corrected path

datagen = ImageDataGenerator(rescale=1./255)
train_data = datagen.flow_from_directory(train_dir, target_size=(128,128), batch_size=32, class_mode='binary')
val_data = datagen.flow_from_directory(val_dir, target_size=(128,128), batch_size=32, class_mode='binary')

# 2. Build Lightweight CNN Model
model = keras.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, validation_data=val_data, epochs=5)

# 3. Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save model
with open('recycle_classifier.tflite', 'wb') as f:
    f.write(tflite_model)

Found 22564 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
706/706 ━━━━━━━━━━━━━━━━━━━━ 145s 199ms/step - accuracy: 0.7828 - loss: 0.5123 - val_accuracy: 0.8818 - val_loss: 0.3330
Epoch 2/5
706/706 ━━━━━━━━━━━━━━━━━━━━ 34s 48ms/step - accuracy: 0.8483 - loss: 0.3564 - val_accuracy: 0.8707 - val_loss: 0.3397
Epoch 3/5
706/706 ━━━━━━━━━━━━━━━━━━━━ 33s 47ms/step - accuracy: 0.8775 - loss: 0.2955 - val_accuracy: 0.8822 - val_loss: 0.2994
Epoch 4/5
706/706 ━━━━━━━━━━━━━━━━━━━━ 33s 47ms/step - accuracy: 0.9016 - loss: 0.2387 - val_accuracy: 0.8818 - val_loss: 0.3325
Epoch 5/5
706/706 ━━━━━━━━━━━━━━━━━━━━ 34s 49ms/step - accuracy: 0.9313 - loss: 0.1739 - val_accuracy: 0.8886 - val_loss: 0.3296
Saved artifact at '/tmp/tmpavngznfz'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  134504692083280: TensorSpec(shape=(), dtype=tf.resource, name=None)

In [4]:
import os

# The train_dir is already defined from the previous cells.
# In your current setup, it is: '/kaggle/input/waste-classification-data/TRAIN'
print(f"Path to the main training directory: {train_dir}")

# List all items (subdirectories and files) within the training directory
print("Contents of the training directory:")
for item in os.listdir(train_dir):
    print(f"- {item}")

# Example: If there's a subdirectory named 'plastic' inside 'TRAIN',
# you can access its path like this:
# plastic_dir = os.path.join(train_dir, 'plastic')
# print(f"Path to the 'plastic' subdirectory: {plastic_dir}")

# To see the contents of a specific subdirectory (e.g., 'plastic' if it exists):
# if 'plastic' in os.listdir(train_dir):
#     print("\nContents of the 'plastic' subdirectory:")
#     for item in os.listdir(os.path.join(train_dir, 'plastic')):
#         print(f"  - {item}")
# else:
#     print("\n'plastic' subdirectory not found in training data.")

Path to the main training directory: /kaggle/input/waste-classification-data/DATASET/TRAIN
Contents of the training directory:
- R
- O


In [5]:
# Install the ai_edge_litert package
!pip install ai-edge-litert


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 MB 11.4 MB/s eta 0:00:00


In [6]:
import numpy as np
from PIL import Image
import ai_edge_litert.interpreter as tflite  # New import for LiteRT
import os

# Instantiate the LiteRT interpreter with the TFLite model
interpreter = tflite.Interpreter(model_path="recycle_classifier.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Use the same dummy image as before if it exists, or create one.
if not os.path.exists("plastic_bottle.jpg"):
    dummy_image = Image.new('RGB', (128, 128), color = 'red')
    dummy_image.save("plastic_bottle.jpg")

image = Image.open("plastic_bottle.jpg").resize((128,128))
input_data = np.expand_dims(np.array(image, dtype=np.float32)/255.0, axis=0)

interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

print("Prediction using LiteRT:", "Recyclable" if output_data[0]>0.5 else "Non-Recyclable")


Prediction using LiteRT: Recyclable


In [7]:
import numpy as np
from PIL import Image
import tensorflow.lite as tflite # Changed import from tflite_runtime.interpreter

interpreter = tflite.Interpreter(model_path="recycle_classifier.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Ensure 'plastic_bottle.jpg' exists or provide a placeholder image.
# For demonstration, let's create a dummy image if it doesn't exist.
import os
if not os.path.exists("plastic_bottle.jpg"):
    dummy_image = Image.new('RGB', (128, 128), color = 'red')
    dummy_image.save("plastic_bottle.jpg")

image = Image.open("plastic_bottle.jpg").resize((128,128))
input_data = np.expand_dims(np.array(image, dtype=np.float32)/255.0, axis=0)

interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Prediction:", "Recyclable" if output_data[0]>0.5 else "Non-Recyclable")

Prediction: Recyclable


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
